In [47]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings('ignore')

In [48]:
datas=pd.read_csv('output.csv')

In [49]:
mood_data=pd.read_csv('mood_data.csv')

In [6]:
datas.head(5)

,adult,genres,movieId,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,runtime,spoken_languages,tagline,title,cast,crew,keywords
0,False,DramaRomance,79782,en,Wenecja,An atmospheric coming-of-age story featuring a...,0.147130,NaN,NaN,2010-05-25,110.0,plPolski,NaN,Venice,Marcin WalewskiMagdalena CieleckaMariusz Bonas...,Jan Jakub KolskiArtur ReinhartWitold Chominski...,English
1,False,ComedyHorror,141210,en,The Sleepover,"The town of Derry has a secret, but no one tol...",0.135596,NaN,NaN,2013-10-12,6.0,English,NaN,The Sleepover,Josh FeldmanGus KampCarolyn JaniaKristine Blac...,Chris Cullari,English
2,False,Drama,143750,en,The Farmer's Wife,"As her surroundings are invaded by outsiders, ...",0.211754,NaN,NaN,2012-06-20,18.0,enEnglish,NaN,The Farmer's Wife,James CartwrightGeraldine JamesAlex KellyMarti...,John ParishJonas GrimåsGary ShawFrancis LeeFra...,short
3,False,Documentary,84198,en,A Place at the Table,"Using personal stories, this powerful document...",0.501046,NaN,USUnitedStatesofAmerica,2012-03-22,84.0,enEnglish,One Nation. Underfed.,A Place at the Table,Jeff BridgesTom ColicchioMariana ChiltonKen Co...,Kristi JacobsonLori Silverbush,woman director
4,False,Documentary,84198,en,A Place at the Table,"Using personal stories, this powerful document...",0.501046,NaN,USUnitedStatesofAmerica,2012-03-22,84.0,enEnglish,One Nation. Underfed.,A Place at the Table,Jeff BridgesTom ColicchioMariana ChiltonKen Co...,Kristi JacobsonLori Silverbush,woman director


In [7]:
vectorizer = TfidfVectorizer()

In [8]:
# Replace NaN values with "unknown"
datas['genres'].fillna("unknown", inplace=True)

In [9]:
feature_vectors_genres= vectorizer.fit_transform(datas['genres'])

In [10]:
feature_vectors_genres

<22176x2486 sparse matrix of type '<class 'numpy.float64'>'
	with 22176 stored elements in Compressed Sparse Row format>

In [11]:
feature_vectors_overview= vectorizer.fit_transform(datas['overview'])

In [12]:
feature_vectors_overview

<22176x52357 sparse matrix of type '<class 'numpy.float64'>'
	with 945355 stored elements in Compressed Sparse Row format>

In [13]:
feature_vectors_language= vectorizer.fit_transform(datas['spoken_languages'])

In [14]:
feature_vectors_language

<22176x1378 sparse matrix of type '<class 'numpy.float64'>'
	with 22176 stored elements in Compressed Sparse Row format>

In [15]:
# Replace NaN values with "unknown"
datas['cast'].fillna("unknown", inplace=True)

In [16]:
feature_vectors_cast= vectorizer.fit_transform(datas['cast'])

In [17]:
feature_vectors_cast

<22176x241425 sparse matrix of type '<class 'numpy.float64'>'
	with 312352 stored elements in Compressed Sparse Row format>

In [18]:
datas['crew'].fillna("unknown", inplace=True)

In [19]:
feature_vectors_crew= vectorizer.fit_transform(datas['crew'])

In [20]:
feature_vectors_crew

<22176x212057 sparse matrix of type '<class 'numpy.float64'>'
	with 288511 stored elements in Compressed Sparse Row format>

In [21]:
similarity_genres = cosine_similarity(feature_vectors_genres)

In [22]:
similarity_genres 

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 1., 0., 0.],
       ...,
       [0., 0., 1., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [23]:
similarity_overview = cosine_similarity(feature_vectors_overview)

In [24]:
similarity_language = cosine_similarity(feature_vectors_language)

In [25]:
similarity_cast = cosine_similarity(feature_vectors_cast)

In [26]:
similarity_crew = cosine_similarity(feature_vectors_crew)

In [23]:
'''feature_vectors_keywords= vectorizer.fit_transform(datas['keywords'])'''

In [24]:
'''feature_vectors_keywords'''

<22176x27681 sparse matrix of type '<class 'numpy.float64'>'
	with 57199 stored elements in Compressed Sparse Row format>

In [26]:
'''similarity_keywords = cosine_similarity(feature_vectors_keywords)'''

'similarity_keywords = cosine_similarity(feature_vectors_keywords)'

In [28]:
index=pd.Series(datas.index,index=datas['title']).drop_duplicates()

In [29]:
index

title
Venice                      0
The Sleepover               1
The Farmer's Wife           2
A Place at the Table        3
A Place at the Table        4
                        ...  
Mom                     22171
The Morning After       22172
Century of Birthing     22173
Betrayal                22174
Queerama                22175
Length: 22176, dtype: int64

In [ ]:
mood_data.set_index('USERID', inplace=True)

In [79]:
def recommend(USERID,mood, cosine_sim1=similarity_genres,cosine_sim2=similarity_overview,cosine_sim3=similarity_language,cosine_sim4=similarity_cast,cosine_sim5=similarity_crew):

    title=mood_data.loc[USERID,mood]
    
    '''if title not in index:
        print(f"Title '{title}' not found in the dataset.")
        return []'''

    idx = index[title]
    similarity_score_1= list(enumerate(cosine_sim1[idx]))
    similarity_score_2= list(enumerate(cosine_sim2[idx]))
    similarity_score_3= list(enumerate(cosine_sim3[idx]))
    similarity_score_4= list(enumerate(cosine_sim4[idx]))
    similarity_score_5= list(enumerate(cosine_sim5[idx]))
    
    sorted_similar_movies_1 = sorted(similarity_score_1, key=lambda x: x[1], reverse=True)
    sorted_similar_movies_1= sorted_similar_movies_1[1:10]

    sorted_similar_movies_2 = sorted(similarity_score_2, key=lambda x: x[1], reverse=True)
    sorted_similar_movies_2= sorted_similar_movies_2[1:10]

    sorted_similar_movies_3 = sorted(similarity_score_3, key=lambda x: x[1], reverse=True)
    sorted_similar_movies_3= sorted_similar_movies_3[1:10]

    sorted_similar_movies_4 = sorted(similarity_score_4, key=lambda x: x[1], reverse=True)
    sorted_similar_movies_4 = sorted_similar_movies_4[1:10]

    sorted_similar_movies_5 = sorted(similarity_score_5, key=lambda x: x[1], reverse=True)
    sorted_similar_movies_5 = sorted_similar_movies_5[1:10]

    
    sim_idx_1= [i[0] for i in sorted_similar_movies_1]
    sim_idx_2= [i[0] for i in sorted_similar_movies_2]
    sim_idx_3= [i[0] for i in sorted_similar_movies_3]
    sim_idx_4= [i[0] for i in sorted_similar_movies_4]
    sim_idx_5= [i[0] for i in sorted_similar_movies_5]

    
    mov_genres = datas[['movieId', 'title']].iloc[sim_idx_1]
    mov_overview = datas[['movieId', 'title']].iloc[sim_idx_2]
    mov_language= datas[['movieId', 'title']].iloc[sim_idx_3]
    mov_cast = datas[['movieId', 'title']].iloc[sim_idx_4]
    mov_crew = datas[['movieId', 'title']].iloc[sim_idx_5]

    mov=pd.concat([mov_genres, mov_overview, mov_language, mov_cast, mov_crew], axis=0)
    mov.reset_index(drop=True, inplace=True)
    return mov

In [84]:
recommend(564,'Peace')

,movieId,title
0,24,Kill Bill: Vol. 1
1,314,Catwoman
2,62276,Bummer
3,10027,Unleashed
4,8292,Four Brothers
5,9923,Domino
6,13056,Punisher: War Zone
7,23483,Kick-Ass
8,37766,Uro
9,51875,Miss Nobody


In [50]:
mood_data.head(5)

,USERID,Happy,Sad,Thrilled,Bored,Angry,Terror,Love,Peace,Disgust
0,564,Saw,Hotel Rwanda,Insomnia,Italian for Beginners,Oliver Twist,Snakes on a Plane,Snow Cake,Catwoman,Donnie Darko
1,547,Harry Potter and the Philosopher's Stone,Cars,Mulholland Drive,Men in Black II,Ice Age: The Meltdown,Blood: The Last Vampire,Cinderella Man,Finding Neverland,Dancer in the Dark
2,15,Human Nature,Almost Famous,Tough Enough,The Hours,Blood: The Last Vampire,Aliens vs Predator: Requiem,Monsoon Wedding,Popular Music,The Day After Tomorrow
3,73,Dave Chappelle's Block Party,Dawn of the Dead,Snatch,Together,Cockles and Muscles,Saw II,Notes on a Scandal,Silent Hill,Lost in Translation
4,452,Love Actually,Ice Age: The Meltdown,Ocean's Eleven,Inside Man,Monsieur Ibrahim,Austin Powers in Goldmember,The Elementary Particles,The Prestige,Pirates of the Caribbean: The Curse of the Bla


In [62]:
mood_data.set_index('USERID', inplace=True)

In [63]:
mood_data

,Happy,Sad,Thrilled,Bored,Angry,Terror,Love,Peace,Disgust
USERID,,,,,,,,,
564,Saw,Hotel Rwanda,Insomnia,Italian for Beginners,Oliver Twist,Snakes on a Plane,Snow Cake,Catwoman,Donnie Darko
547,Harry Potter and the Philosopher's Stone,Cars,Mulholland Drive,Men in Black II,Ice Age: The Meltdown,Blood: The Last Vampire,Cinderella Man,Finding Neverland,Dancer in the Dark
15,Human Nature,Almost Famous,Tough Enough,The Hours,Blood: The Last Vampire,Aliens vs Predator: Requiem,Monsoon Wedding,Popular Music,The Day After Tomorrow
73,Dave Chappelle's Block Party,Dawn of the Dead,Snatch,Together,Cockles and Muscles,Saw II,Notes on a Scandal,Silent Hill,Lost in Translation
452,Love Actually,Ice Age: The Meltdown,Ocean's Eleven,Inside Man,Monsieur Ibrahim,Austin Powers in Goldmember,The Elementary Particles,The Prestige,Pirates of the Caribbean: The Curse of the Bla
468,Cockles and Muscles,The Science of Sleep,Syriana,Batman Begins,Tough Enough,28 Days Later,Silent Hill,Whale Rider,Zatoichi
624,Adam's Apples,Stranger Than Fiction,Dawn of the Dead,Dreamgirls,Dawn of the Dead,Ghost Rider,The Dreamers,Bridge to Terabithia,Street Kings
311,As It Is in Heaven,Summer Storm,Dogville,The Matrix Revolutions,Underworld,The Dark Knight,Bread and Tulips,The Terminal,Big Fish
509,Harry Potter and the Goblet of Fire,Borat: Cultural Learnings of America for Make,Planet of the Apes,Blood: The Last Vampire,Inside Man,The Omen,The Flintstones in Viva Rock Vegas,Finding Neverland,Million Dollar Baby
